# Automated Gleason Grading

In [1]:
from lib.load import *
from lib.merged_training import *
from lib.testing import *
from lib.segmented_patches import *

In [2]:
# train_img_names, val_img_names, test_img_names, thumbnails_dict = load()

In [3]:
# unet = mergedTraining(train_img_names, val_img_names, thumbnails_dict)

In [4]:
# Serialize the model to disk
# torch.save(unet, MODEL_PATH)

In [5]:
# testingLoss = testing(unet, test_img_names)

In [6]:
create_segmented_patches()

[INFO] load up model...


  0%|          | 0/3 [00:00<?, ?it/s]

0005f7aaab2800f6170c399693a96917
[3072, 3072]


  0%|          | 0/3 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 3)